# Simple Use of RNA Expression

# RNA Expression

RNASeq
 * Convert RNA to cDNA Library
 * Use NGS sequencing on cDNA
 * Allows a reading of mRNA transcripts and Expression levels
 
 
With RNA sequencing we have the promise of a measurement that should help us understand the actions and reactions of a cell.

## Promise of Reading Transcripts
One of the findings of the genomic analysis found less protein coding differences than originally expected.  Instead RNA expression levels and interactions appear to be major difference between mammals.

Identified as a major factor in human evolution between humans and apes.

<img src="img/chimp.jpg" alt="Drawing" style="width: 200px;"/>

## Promise of Reading Transcripts

#### Gene expression differences contribute to human evolution
 * 'there are significant differences in how genes are expressed and regulated'
 * '.. those differences are most marked in the brain'
 * https://science.sciencemag.org/content/292/5514/44
 * '.. rapid change, along the five million years of the human lineage, that was concentrated on these specific groups of genes.'

 * https://www.scientificamerican.com/article/separation-of-man-and-ape/

#### RNA expression is involved directly or indirectly in most cellular interactions.

## Growing Number of Resources

GTEx v7 has over 15k samples
 * 53 non-diseased tissue sites
 * nearly 1000 indivduals
 * v8 to come out this month
 
TCGA
 * RNA-Seq data for 9736 tumor samples across 33 cancer types
 * data for 726 adjacent normal tissues
 
GSC
 * Over 600 cases from POG at GSC
 * Growing number of literature references in Knowledgebase

## Some GSC uses of mRNA Expression

### Cibersort

Uses patters of mRNA expression levels to quantify immune cell

![cibersort background](img/cibersort_background.png)

https://www.nature.com/articles/nmeth.3337

## GSC RNA Expression Uses

Expression plots for Analysis
 * Visual analysis

### Box_wiskers - Individual Genes over Cancer
<img src="img/box_whisker.png" alt="Drawing" style="height: 400px;"/>


## GSC RNA Expression Uses

### Box_wiskers - Individual Genes over Cancer

![box_whisker](img/box_whisker.png)

## GSC RNA Expression Uses

### Spearman Correlation Against Known Cancer Types

Identify cancer types by gene expression patterns using LOGANOVA spearman correlations.  By C Chng and P Eirew

![Spearman correlation plot](img/Spearman_tcga.png)

### Moffitt Score - Pancreatic Subtype Classifier - New Tool

Determines a probability that a Pancreatic Cancer is of a 'basal' vs 'classic' subtype.

Compares the Expression Levels of a small set of genes:
 * Just compares if Gene A expressed higher than Gene B - unitless
 * Each True value increase basal probability by a pair-specific weight
 
 ```
     Coefficient    Gene_A   Gene_B  basal_odds_increase
0          0.87     CD109   GPR160                 2.38
1          1.22    SLC2A1     AGR2                 3.39
2          0.52     KRT16  SLC44A4                 1.68
3          1.43      CTSV  TMEM45B                 4.17
4          0.70     KRT6A    BCAS1                 2.01
5          0.41    B3GNT5    VSIG2                 1.51
6          0.72       MET     TFF3                 2.06
7          0.80     CHST6  PLA2G10                 2.24
8          0.76  SERPINB5     HPGD                 2.13
9          1.40    DCBLD2     PLS1                 4.07
10         1.33    IL20RB    FAM3D                 3.79
11         1.58  PPP1R14C    SYTL2                 4.85
12         0.41      NAB1  PLEKHA6                 1.50
13         1.58      MSLN    CAPN9                 4.83
```
 https://www.ncbi.nlm.nih.gov/pubmed/26343385
 

![moffit exampl](img/moffitt_hist.png)
 

![moffit scatter](img/moffitt_vs_GATA6.png)

## GSC RNA Expression Uses

Expression plots for Analysis
 * Visual analysis

### Within Project Correlation (WPC)

![WPC](img/WPC.png)

## GSC RNA Expression Uses

### Report Outliers and KB matching

#### Goals
For Reporting, we want to highlight relevant gene expressions.
 * Expressions with therapeutic or diagnositic significance
 * Expressions supporting other events, like copy number change
 * Expression levels similar to known loss-of-function or gain-of-function
 * Unusual, cancer-related gene expressions
 * Features, like activated pathways

### Report Outliers and KB matching
#### Current Method - Crude cutoffs

Currently we use a number of cutoffs, depending on available data, to try to flag expression as relevant.
 
For High Expression:
 * Higher expression than normal data:
  * 2 x illumina bodymap expression comparator
  * > TCGA adjacent normal (if sig. number available)
 * TCGA top quartile
  * > 2xinter-quartile ranges ~ top 1% of proper normal distribution

## TF4CN - COMPARISON Problem Case

### External Expression Data Gave Strange Results

*Report outliers flagged too many genes*

But Spearman Correlation still very effectively identified Colorectal expression patterns


 * What is going wrong with the outlier calls?
 * Why does the Spearman Correlation still work?
 * How could outlier calling be improved?
 
 ![Comparison prob](img/TF4CN_percentile.png)

# mRNA Expression
## Expression Level Differences can complicated

At a basic level mRNA expression is expected to correlate with protein expression.

### Roughly Correlates to Protein Levels

![Variations in mRNA vs Protein levels](img/rna_vs_protein.png)

from: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4459080/


### Roughly Correlates to Protein Levels - Different Genes Vary
 * [mRNA] / [protein] ratio varies by several orders of magnitude across genes.
 * but is generally consistent across cells and tissues, for each gene/protein.
  * Quantifying gene expression: the importance of being subtle
  * https://www.embopress.org/doi/pdf/10.15252/msb.20167325

![Variations in mRNA vs Protein levels](img/Gene_expression_subtle.png)

### Small differences can have a large effect

What is a 'significant' difference in expression?

Classic blond hair vs brunette in europeans is caused by 20% decrease in mRNA expression.
<img src="img/blond.jpg" alt="Drawing" style="width: 200px;"/>

 * Caused by a SNP 350 kb away from KITLG coding region
 * SNP alters transcription factor binding
 * only 20% change is less than most check for significance.
 * https://www.nature.com/articles/ng.3019 Guether et al. 2014
 


### Population at large can have outliers
Can be large differences in normal population.
 * The impact of rare variation on gene expression across tissues
  * 58% of underexpression and 28% of overexpression outliers have nearby conserved rare variants compared to 8% of non-outliers
  * bayesian improved calling
 * https://www.ncbi.nlm.nih.gov/pubmed/29022581

# Good Comparisons are Required

Most uses of expression data will use a comparison of the gene expression pattern to known references.
What would this involve?

## What does Typical Expression Look like

Genotype-Tissue Expression (GTEx) project is becoming a major resource for normal human tissue.
 * v8 to be released this month.

The spread includes extreme values, most of the tissues are similar.

Mean is ~ 16 - 17 TPM, but 50th percentile is 0 and 75th percentile is ~ 2.

This indicates that the TPM is not a normal distribution, but has a small number of extreme values.  Almost half of the 56K gene set show no expression.

```
  Unnamed: 0  Adipose Tissue  Adrenal Gland       Bladder          Blood
0      count    56202.000000   56202.000000  56202.000000   56202.000000
1       mean       16.335619      17.107780     16.663300      15.233666
2        std      398.650451     582.642470    351.055527    1113.018285
3        min        0.000000       0.000000      0.000000       0.000000
4        25%        0.000000       0.000000      0.000000       0.000000
5        50%        0.000000       0.000000      0.000000       0.000000
6        75%        1.897000       1.492375      2.668000       0.492100
7        max    38300.000000   60775.000000  34110.000000  246600.000000
```
 * Spearman Correlation is removing all RPKM < 0.2

## Natural Distribution is not a Normal distribution

This means the log of the values shows a normal distribution.

### PTEN shows a Skewed Curve over GTEX

![](img/PTEN_raw.png)

### PTEN shows the normal distribution after log transform
![](img/PTEN_log2.png)

### What's happening with TP53

What does the camel like curve mean?

![](img/TP53_raw.png)

### What's happening with TP53

What does the camel like curve mean?
![](img/TP53_log2.png)

### What's happening with TP53 - Different Expression by tissue

![](img/TP53_by_tissue.png)

GTEx shows the range of expressions and importance of tissue effects.

# Reference Problems

Directly trying to compare GTEx and TCGA Expression can result in problems - units and experimental differences.

### Units RPKM, FPKM, TPM

 * RPKM (single-end data) - Reads Per Kilobase of transcript per Million mapped reads
 * FPKM (paired-end data) - Fragments Per Kilobase of transcript per Million mapped reads
 * TPM  - Transcripts Per Million RNA molecules

RPKM and FPKM are normalized within a sample

TPM is more normalized for comparing between samples

### Units RPKM, FPKM, TPM
Convert to TPM simply by dividing each RPKM value by the sum of the RPKM values for all genes (or transcripts) and multiplying by one million.

TPM = (mean transcript length in kilobases) x RPKM 
where "mean transcript length" is the expression-weighted mean of the lengths of all isoforms.  Because the mean transcript length can change from sample to sample, we have generally recommended the use of TPM instead of RPKM.

If you have RPKM (single-end data) or FPKM (paired-end data) computed for a set of genes or transcripts you can convert to TPM with 

TPM = FPKM / (sum of FPKM over all genes/transcripts) * 10^6

Since TPM is independent of the mean transcript length, it should be more comparable between samples

# Reference Problems
### Gene Models

 * Definitions of Gene / Transcript important for what is measured
  * Ensembl69 vs Ensembl 74 for TF4CN
 * Also - what transcript is chosen
  * collapsed model of all transcripts vs longest transcript
  
## Why was Spearman Correlation Robust to Gene Models?

## Spearman vs Pearson correlation
![Spearman correlation uses rank order](img/spearman_vs_pearson_1.png)

## Spearman vs Pearson correlation
![Spearman correlation shows a better relationship for curves](img/spearman_vs_pearson_2.png)

## Spearman vs Pearson correlation
![Spearman only measures monotonic - both fail](img/spearman_vs_pearson_3.png)

## Expression Patterns Still Correlate

GTEx histograms of PTEN, with and without correction for GSC gene models and units.

![model](img/PTEN_ENSG00000171862.png)

## TF4CN Expression Outlier Post-mortem

Among the problems identified were:
 * Different Gene model - effects only some genes
  * Ensembl 74
  * Longest Transcript vs collapsed model
 * Protocol difference - Ribodepleted vs PolyA
  * Different extreme outliers based on read count distribution
 * Outlier cutoffs were not using log transform
  * unlike Spearman Cancer types
  * creates more outliers.
 * small number of illumnia reference values vs GTEx set
  * makes identifying unusual behaviour much more difficult

#### An Example of the greater problem

"RNA-seq expression levels of the majority of genes quantified are in the range of 4-10 (log2 of normalized_count) for TCGA, and 0-4 (log2 of RPKM) for GTEx,a consequence of the use of different analysis pipelines. 

This makes gene expression levels from the two projects not directly comparable."
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5903355/

# New Outlier Detection Methods

Need methods to flag cancer related expression outliers, separate from confounding effects like gene models or RNAseq protocol.

## PEER - Probabilistic Estimation of Expression Residuals

 * Used by GTEx
 * Bayesian framework to account for complex non-genetic factors in gene expression levels.
 
![PEER factors](img/PEER_factors.png)
 
https://www.sanger.ac.uk/science/tools/peer

## PEER - Probabilistic Estimation of Expression Residuals

Bayesian Method assigns probabilities and can be trained with labelled data:
 * Probability difference is experimental (Ribodepleted vs PolyA)
 * Probability of effect being FFPE processing
 * Probability of tumour content effect
 * Probability of gene being a cancer driver
 * Probability of drug response

## OUTRIDER - Outlier in RNA-Seq Finder

ML Autoencoder trained on typical gene covariations

![outrider](img/OUTRIDER.png)
https://doi.org/10.1016/j.ajhg.2018.10.025

# Current Work

## Make Use of GTEx Normals and Z-scores

 * GTEx offers a drastic increase in number of samples for comparison
  * illumina bodymap ~ 1 reference for 16 tissues
  * GTEx over 100 samples for > 30 tissues
 * to compare to a set of GTEx normals use:
  * percentile rank
  * Z-scores

## What is a Z-score?
 * Literature often refers to Z-score values
 * Preferred unit for cBioPortal expression
 
 Just means:
 #### Number of Standard Deviations away from Mean
  * Z-score +2.5: Value is 2.5 std. above mean
  * Z-score -1: Value is 1 std. below mean
  * Z-score 0: Average

## Why use a Z-score?
z = (x-mean)/std

### Assuming the Reference is Normally Distributed
 * Standard statistical meaning:
  * ~95% within 2 std of mean
  * 99.7 within 3 std of mean
 * has no units:
  * Compare between any types of experiments.

## Using Z-score
Statistical assumptions work better the more normalized the reference data
![](img/PTEN_raw.png)

## Using Z-score
Meaning is related to the quality of the reference
![](img/TP53_by_tissue.png)

## Make Use of GTEx Z-scores

Even simple statistics can flag outlier genes:
 * normal distribution deviance by skew and kurtosis
 * Chronic low / high vs our measurement - poor comparsion
  * gene model problem
  * experimental differences

# Current Work

## TMM -  trimmed mean of M-values normalization
 * per sample normalization
 * Assume most genes have standard expression
 * Ignore top / bottom quartile of differences
 * weighted fit of remaing genes

# Current Work

## TMM-like -  Advanced
 * Some genes will have more consistent expression
  * adjust fit weights to reflect this
  * keep signal by omitting cancer genes from fitting

#  Concepts Going Forward

 * Use log transform for statistics / machine learning
 * like mRNA -> protein expression, most relationships are probably gene specific
  * mRNA to protein response is probably significant
  * eg. Only some genes change with Ensembl version difference
  * indivualistic responses to ribodepletion
 * Individual genes can be hit by experimental / modelling differences
  * comparison to groups or scores of groups are more robust
 * Gene expression is highly correlated
 * Most genes express at very low levels

# Thank You

#### Many thanks to Karen Mungall

#### All of the Clinical Informatics Team
 * For answering my many, many questions about expression

#### Bioinformatics and PIPE
 * For supplying all many types of data

#### SDEV
 * Great work by team members - letting me move on to this work

#### Steve Jones Group
 * For ideas and advice

## Going Forward

### Statistical and ML Models
 * Pathway activation


### Experimental Differences

 * Gene models used
  * Ensembl69 vs Ensembl75 vs Ensembl92
  * Collapsed model, or individual transcripts?
 * Ribodepleted
 
 Other complications:
  * known factors - age, gender
  * sample treatment - FFPE samples
  * tumour content - mixtures
  * hidden factors - environment, temperature

# Expression Effects

## Genes are highly correlated in expression

 * Expression as a pathway property
 * Altered pathway expression - re-wiring concept
 * Measure metabolite(s) cycle activity
 * Measure Transcription Factor Levels
 * Other genomic influences:
  * Many SNPs, copy number changes alter expression
  * Driver fusions creating inappropriate activity

In [3]:
from gtex import GTEX_preprocess
importlib.reload(GTEX_preprocess)
gtex = GTEX_preprocess.GtexData()


2019-08-22 12:02:03 - root - INFO - logging
2019-08-22 12:02:03 - root - INFO - logging
2019-08-22 12:02:03 - root - INFO - Found: /home/dbleile/science/Xpress/gtex/data_cache/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct
2019-08-22 12:02:03 - root - INFO - Found: /home/dbleile/science/Xpress/gtex/data_cache/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct
2019-08-22 12:02:03 - root - INFO - Found: /home/dbleile/science/Xpress/gtex/data_cache/GTEx_v7_Annotations_SampleAttributesDS.txt
2019-08-22 12:02:03 - root - INFO - Found: /home/dbleile/science/Xpress/gtex/data_cache/GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx
2019-08-22 12:02:03 - root - INFO - Found: /home/dbleile/science/Xpress/gtex/data_cache/GTEx_v7_Annotations_SubjectPhenotypesDS.txt


In [4]:
tp53 = gtex.gtex_gene_table(['TP53', 'PTEN'])

2019-08-22 12:02:03 - root - INFO - Loading reference GTEX7 row table
2019-08-22 12:02:34 - root - INFO - loading 3 GTEX TPM rows
2019-08-22 12:02:44 - root - INFO - Gtex TPM partial table took 0.6869680285453796 minutes


In [ ]:
gtpm = gtex.gtex_gene_table(None)

2019-08-22 12:02:44 - root - INFO - loading all rows


In [ ]:
gtpm = gtmp
gtpm.head()

In [1]:
import importlib
import os, sys, glob

import pandas as pd
import numpy as np
import seaborn as sns
import pathlib
script_dir = pathlib.Path().resolve()

gm_fn = os.path.join(script_dir, 'tables', 'GTEX_tissue_medians_desc.tsv')
gm = pd.read_csv(gm_fn, sep='\t')
print(gm[gm.columns[:5]])

  Unnamed: 0  Adipose Tissue  Adrenal Gland       Bladder          Blood
0      count    56202.000000   56202.000000  56202.000000   56202.000000
1       mean       16.335619      17.107780     16.663300      15.233666
2        std      398.650451     582.642470    351.055527    1113.018285
3        min        0.000000       0.000000      0.000000       0.000000
4        25%        0.000000       0.000000      0.000000       0.000000
5        50%        0.000000       0.000000      0.000000       0.000000
6        75%        1.897000       1.492375      2.668000       0.492100
7        max    38300.000000   60775.000000  34110.000000  246600.000000


In [2]:
gmo_fn = os.path.join(script_dir, 'tables', 'GTEX_tissue_medians_onco_ts.tsv')
gmo = pd.read_csv(gmo_fn, sep='\t')
print(gmo[gmo.columns[:5]])

  Unnamed: 0  Adipose Tissue  Adrenal Gland      Bladder        Blood
0      count      779.000000     779.000000   779.000000   779.000000
1       mean       50.191434      33.618767    49.578746    50.097736
2        std      158.616941     101.872038   131.885045   360.652820
3        min        0.000000       0.000000     0.000000     0.000000
4        25%        4.507500       2.593250     5.602000     0.576500
5        50%       14.680000      10.005000    18.820000     5.052000
6        75%       33.765000      25.515000    41.905000    20.595000
7        max     1979.000000    1084.500000  1770.000000  8644.000000
